In [3]:
pip install ultralytics

  Using cached ultralytics-8.3.94-py3-none-any.whl.metadata (35 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
Using cached ultralytics-8.3.94-py3-none-any.whl (949 kB)
Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
import cv2
import torch
import numpy as np
import torchvision.transforms as transforms
from ultralytics import YOLO
from PIL import Image
import torchvision.models as models
import torch.nn as nn

In [8]:

# Load model YOLO
yolo_model = YOLO("bestYolo11.pt")  # Ganti dengan model YOLO yang telah dilatih

# Load CNN EfficientNet-B0 dengan state_dict
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cnn_model = models.efficientnet_b0(pretrained=False)
cnn_model.classifier[1] = nn.Linear(cnn_model.classifier[1].in_features, 2)  # Pastikan output 2 kelas
cnn_model.load_state_dict(torch.load("best_model_efficientnet.pth", map_location=device))
cnn_model.to(device).eval()  # Set model ke mode evaluasi

# Definisi transformasi gambar untuk CNN
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize ke ukuran yang sesuai dengan model CNN
    transforms.ToTensor(),  # Konversi ke tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisasi
])

# Label CNN
cnn_labels = ["Memakai-Helm", "Tidak-Memakai-Helm"]
path = "D:\\TA_Lalu_Lintas\\TA-Lalu-Lintas\\dataset-mentah\\vidios\\TC_00006.mov"

# Buka video streaming
cap = cv2.VideoCapture(path)  # Gunakan 'video.mp4' jika ingin memakai video file

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Gunakan YOLO untuk deteksi
    results = yolo_model(frame)

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Koordinat bounding box
            confidence = float(box.conf[0])  # Confidence score
            class_id = int(box.cls[0])  # ID class YOLO

            # Hanya proses jika kelas yang terdeteksi adalah 'pengendara tanpa helm'
            if class_id == 1 and confidence > 0.7:  
                # Pastikan bounding box tidak keluar dari frame
                height, width, _ = frame.shape
                x1, y1, x2, y2 = max(0, x1), max(0, y1), min(width, x2), min(height, y2)

                cropped_img = frame[y1:y2, x1:x2]  # Crop gambar hasil deteksi
                
                # Pastikan gambar tidak kosong
                if cropped_img.shape[0] > 0 and cropped_img.shape[1] > 0:
                    # Convert cropped image ke format yang bisa digunakan oleh CNN
                    img_pil = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
                    img_tensor = transform(img_pil).unsqueeze(0).to(device)  # Tambahkan batch dimension dan pindahkan ke GPU

                    # Prediksi menggunakan CNN
                    with torch.no_grad():
                        output = cnn_model(img_tensor)
                        label_index = torch.argmax(output).item()
                        label = cnn_labels[label_index]
                        color = (0, 0, 255) if label_index == 1 else (0, 255, 0)  # Merah untuk pelanggar, hijau untuk tidak

                    # Tambahkan bounding box dan label ke frame
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(frame, f"{label} ({confidence:.2f})", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Tampilkan hasil deteksi pada video streaming
    cv2.imshow("E-Tilang YOLO + CNN", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



WARNING  NMS time limit 2.050s exceeded
0: 384x640 1 Pengendara, 1804.1ms
Speed: 2412.7ms preprocess, 1804.1ms inference, 4521.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 67.6ms
Speed: 48.5ms preprocess, 67.6ms inference, 217.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 67.1ms
Speed: 4.3ms preprocess, 67.1ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 66.5ms
Speed: 4.3ms preprocess, 66.5ms inference, 62.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 67.5ms
Speed: 107.9ms preprocess, 67.5ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 68.5ms
Speed: 7.6ms preprocess, 68.5ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 69.1ms
Speed: 8.0ms preprocess, 69.1ms inference, 2.5ms postprocess per image at shape (1,

In [1]:
# best
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from ultralytics import YOLO
from PIL import Image
import os
import numpy as np

# Load model YOLO
yolo_model = YOLO("bestYolo11.pt")  # Ganti dengan model YOLO yang telah dilatih

# Load CNN EfficientNet-B0 dengan state_dict
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cnn_model = models.efficientnet_b0(pretrained=False)
cnn_model.classifier[1] = nn.Linear(cnn_model.classifier[1].in_features, 2)  # Output 2 kelas
cnn_model.load_state_dict(torch.load("best_model_efficientnet.pth", map_location=device))
cnn_model.to(device).eval()  # Set model ke mode evaluasi

# Definisi transformasi gambar untuk CNN
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize ke ukuran yang sesuai dengan model CNN
    transforms.ToTensor(),  # Konversi ke tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisasi
])

# Label CNN
cnn_labels = ["Memakai-Helm", "Tidak-Memakai-Helm"]

# Path video
video_path = "D:\\TA_Lalu_Lintas\\TA-Lalu-Lintas\\dataset-mentah\\vidios\\TC_00006.mov"

# Folder untuk menyimpan pelanggar
save_dir = "pelanggar"
os.makedirs(save_dir, exist_ok=True)  # Buat folder jika belum ada

# Buka video streaming
cap = cv2.VideoCapture(video_path)

# Simpan daftar pengendara yang sudah terdeteksi untuk menghindari deteksi berulang
previous_riders = []

def calculate_iou(box1, box2):
    """Hitung Intersection over Union (IoU) antara dua bounding box"""
    x1, y1, x2, y2 = box1
    x1_p, y1_p, x2_p, y2_p = box2

    # Hitung luas intersection
    xi1 = max(x1, x1_p)
    yi1 = max(y1, y1_p)
    xi2 = min(x2, x2_p)
    yi2 = min(y2, y2_p)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

    # Hitung luas union
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_p - x1_p) * (y2_p - y1_p)
    union_area = box1_area + box2_area - inter_area

    # Hitung IoU
    iou = inter_area / union_area if union_area != 0 else 0
    return iou

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Gunakan YOLO untuk deteksi
    results = yolo_model(frame)

    detected_riders = []  # Simpan semua pengendara yang terdeteksi
    detected_helmets = []  # Simpan semua helm yang terdeteksi

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Koordinat bounding box
            confidence = float(box.conf[0])  # Confidence score
            class_id = int(box.cls[0])  # ID class YOLO

            # Pastikan bounding box tidak keluar dari frame
            height, width, _ = frame.shape
            x1, y1, x2, y2 = max(0, x1), max(0, y1), min(width, x2), min(height, y2)

            # Simpan data deteksi sesuai class
            if class_id == 1 and confidence > 0.7:  # Kelas "Helm"
                detected_helmets.append((x1, y1, x2, y2))
                color = (255, 255, 0)  # Warna biru untuk helm
                label = "Helm"
            elif class_id == 0 and confidence > 0.7:  # Kelas "Pengendara"
                detected_riders.append((x1, y1, x2, y2))
                color = (0, 255, 255)  # Warna cyan untuk pengendara
                label = "Pengendara"

            # Gambar bounding box untuk deteksi YOLO
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"{label} ({confidence:.2f})", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Proses hanya untuk pengendara yang terdeteksi
    for (x1, y1, x2, y2) in detected_riders:
        # Cek apakah pengendara ini sudah diproses sebelumnya (menghindari deteksi duplikat)
        is_duplicate = any(calculate_iou((x1, y1, x2, y2), prev_box) > 0.5 for prev_box in previous_riders)
        if is_duplicate:
            continue  # Lewati jika sudah dideteksi sebelumnya

        cropped_img = frame[y1:y2, x1:x2]  # Crop seluruh pengendara

        # Pastikan gambar tidak kosong
        if cropped_img.shape[0] > 0 and cropped_img.shape[1] > 0:
            img_pil = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
            img_tensor = transform(img_pil).unsqueeze(0).to(device)  # Konversi ke tensor

            # Prediksi menggunakan CNN
            with torch.no_grad():
                output = cnn_model(img_tensor)
                label_index = torch.argmax(output).item()
                label = cnn_labels[label_index]
                color = (0, 0, 255) if label_index == 1 else (0, 255, 0)  # Merah untuk pelanggar, hijau untuk tidak

            # Tambahkan bounding box dan label ke frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"{label}", (x1, y1 - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Jika CNN menyatakan tidak memakai helm, simpan gambar ke folder "pelanggar"
            if label_index == 1:  # 1 = Tidak Memakai Helm
                save_path = os.path.join(save_dir, f"pelanggar_{x1}_{y1}.jpg")
                cv2.imwrite(save_path, cropped_img)
                print(f"Gambar pelanggar disimpan: {save_path}")

        # Simpan bounding box pengendara untuk frame berikutnya
        previous_riders.append((x1, y1, x2, y2))

    # Tampilkan hasil deteksi pada video streaming
    cv2.imshow("E-Tilang YOLO + CNN", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


d:\ComputerVision\belajarConda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\ComputerVision\belajarConda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



WARNING  NMS time limit 2.050s exceeded
0: 384x640 1 Pengendara, 5510.7ms
Speed: 5316.9ms preprocess, 5510.7ms inference, 6048.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 87.1ms
Speed: 11.0ms preprocess, 87.1ms inference, 70.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 66.3ms
Speed: 2.7ms preprocess, 66.3ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 66.2ms
Speed: 3.5ms preprocess, 66.2ms inference, 34.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 66.5ms
Speed: 3.0ms preprocess, 66.5ms inference, 4.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 66.1ms
Speed: 3.1ms preprocess, 66.1ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 66.3ms
Speed: 3.2ms preprocess, 66.3ms inference, 2.3ms postprocess per image at shape (1, 3,

In [3]:
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from ultralytics import YOLO
from PIL import Image
import os
import numpy as np

# Load model YOLO
yolo_model = YOLO("bestYolo11.pt")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load CNN EfficientNet-B0 dengan state_dict
cnn_model = models.efficientnet_b0(pretrained=False)
cnn_model.classifier[1] = nn.Linear(cnn_model.classifier[1].in_features, 2)
cnn_model.load_state_dict(torch.load("best_model_efficientnet.pth", map_location=device))
cnn_model.to(device).eval()

# Transformasi gambar untuk CNN
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

cnn_labels = ["Memakai-Helm", "Tidak-Memakai-Helm"]
video_path = "D:\\TA_Lalu_Lintas\\TA-Lalu-Lintas\\dataset-mentah\\vidios\\TC_00006.mov"
save_dir = "pelanggar"
os.makedirs(save_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)

MIN_BOX_SIZE = 10000  # Ukuran minimal bounding box agar plat terlihat

detected_riders = []  # Simpan pengendara yang sudah diproses

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = yolo_model(frame)
    
    for result in results:
        for i, box in enumerate(result.boxes):
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = float(box.conf[0])
            class_id = int(box.cls[0])
            
            if class_id == 0 and confidence > 0.7:
                box_area = (x2 - x1) * (y2 - y1)
                
                # Hanya proses bounding box yang cukup besar
                if box_area > MIN_BOX_SIZE:
                    cropped_img = frame[y1:y2, x1:x2]
                    if cropped_img.shape[0] > 0 and cropped_img.shape[1] > 0:
                        img_pil = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
                        img_tensor = transform(img_pil).unsqueeze(0).to(device)
                        
                        with torch.no_grad():
                            output = cnn_model(img_tensor)
                            label_index = torch.argmax(output).item()
                            label = cnn_labels[label_index]
                            
                        if label_index == 1:
                            # Cek apakah pengendara ini sudah terdeteksi sebelumnya
                            rider_box = (x1, y1, x2, y2)
                            if not any(np.array_equal(rider_box, prev_box) for prev_box in detected_riders):
                                save_path = os.path.join(save_dir, f"pelanggar_{x1}_{y1}.jpg")
                                cv2.imwrite(save_path, cropped_img)
                                detected_riders.append(rider_box)
                                print(f"Gambar pelanggar disimpan: {save_path}")
                
                # Gambar bounding box hanya untuk pengendara YOLO dengan ID, confidence, dan label
                label_text = f"ID: {i}, Conf: {confidence:.2f}, Label: Pengendara"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
                cv2.putText(frame, label_text, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
    
    cv2.imshow("E-Tilang YOLO", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 1 Pengendara, 67.3ms
Speed: 2.6ms preprocess, 67.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 91.8ms
Speed: 4.4ms preprocess, 91.8ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 65.9ms
Speed: 3.5ms preprocess, 65.9ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 65.8ms
Speed: 5.5ms preprocess, 65.8ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 65.0ms
Speed: 3.2ms preprocess, 65.0ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 64.0ms
Speed: 3.8ms preprocess, 64.0ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Pengendara, 1 Helm, 64.4ms
Speed: 3.8ms preprocess, 64.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Pengendaras, 1 Helm, 63.7ms


In [4]:
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from ultralytics import YOLO
from PIL import Image
import os
import numpy as np
import datetime
from collections import deque

def remove_noise(image):
    return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)

# Fungsi untuk meningkatkan kontras
def enhance_contrast(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    
    lab = cv2.merge((cl, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

# Fungsi untuk mempertajam gambar
def sharpen_image(image):
    kernel = np.array([[0, -1, 0],
                       [-1, 5,-1],
                       [0, -1, 0]])  # Kernel sharpening
    return cv2.filter2D(image, -1, kernel)

# Konfigurasi
YOLO_MODEL_PATH = "bestYolo11.pt"
CNN_MODEL_PATH = "best_model_efficientnet.pth"
VIDEO_PATH = "D:\\TA_Lalu_Lintas\\TA-Lalu-Lintas\\dataset-mentah\\vidios\\TC_00006.mov"
SAVE_DIR = "pelanggar"
MIN_BOX_AREA = 10000  # Minimum area bounding box (sesuaikan dengan resolusi video)
CONFIDENCE_THRESHOLD = 0.7
COOLDOWN_FRAMES = 100  # Cooldown 30 frame sebelum deteksi ulang objek yang sama
HELMET_LABELS = ["Memakai-Helm", "Tidak-Memakai-Helm"]

# Inisialisasi device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model YOLO
yolo_model = YOLO(YOLO_MODEL_PATH)
yolo_model.to(device)

# Load model EfficientNet-B0
def load_cnn_model():
    model = models.efficientnet_b0(pretrained=False)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    model.load_state_dict(torch.load(CNN_MODEL_PATH, map_location=device))
    return model.to(device).eval()

cnn_model = load_cnn_model()

# Transformasi gambar
cnn_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Persiapan output directory
os.makedirs(SAVE_DIR, exist_ok=True)

class RiderTracker:
    def __init__(self, cooldown):
        self.history = deque(maxlen=100)
        self.cooldown = cooldown
        self.current_id = 0
        
    def is_new_violation(self, box):
        x_center = (box[0] + box[2]) // 2
        y_center = (box[1] + box[3]) // 2
        
        for entry in self.history:
            prev_box, prev_frame = entry
            px = (prev_box[0] + prev_box[2]) // 2
            py = (prev_box[1] + prev_box[3]) // 2
            
            # Hitung jarak Euclidean
            distance = ((x_center - px)**2 + (y_center - py)**2)**0.5
            if distance < 50:  # Threshold jarak dalam piksel
                return False
        return True

    def add_violation(self, box):
        self.history.append((box, self.current_id))
        self.current_id += 1

tracker = RiderTracker(cooldown=COOLDOWN_FRAMES)

def process_detection(frame, box, class_id, confidence):
    x1, y1, x2, y2 = map(int, box)
    box_area = (x2 - x1) * (y2 - y1)
    
    if box_area < MIN_BOX_AREA:
        return None
    
    # Crop dan praproses gambar
    cropped = frame[y1:y2, x1:x2]
    if cropped.size == 0:
        return None
    
    try:
        # Lakukan preprocessing sama seperti data training
        img = cv2.resize(cropped, (224, 224))
        img = remove_noise(img)
        img = sharpen_image(img)
        img = enhance_contrast(img)
        
        # Konversi ke RGB dan PIL Image
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb)
        
        img_tensor = cnn_transform(img_pil).unsqueeze(0).to(device)
    except Exception as e:
        print(f"Error processing image: {e}")
        return None
    
    # Klasifikasi dengan CNN
    with torch.no_grad():
        outputs = cnn_model(img_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        _, preds = torch.max(outputs, 1)
    
    label_index = preds.item()
    label = HELMET_LABELS[label_index]
    
    # Ambil confidence score
    confidence_score = probabilities[0][label_index].item()
    
    # Simpan pelanggaran hanya jika confidence > threshold
    if label_index == 1 and confidence_score > 0.95 and tracker.is_new_violation((x1, y1, x2, y2)):
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S%f")
        save_path = os.path.join(SAVE_DIR, f"violation_{timestamp}.jpg")
        cv2.imwrite(save_path, cropped)
        tracker.add_violation((x1, y1, x2, y2))
        print(f"Pelanggaran terdeteksi: {save_path} (Confidence: {confidence_score:.2f})")
    
    return label, label_index, confidence_score

# Main processing loop
cap = cv2.VideoCapture(VIDEO_PATH)
frame_count = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
    
    frame_count += 1
    results = yolo_model(frame, verbose=False)
    
    # Proses deteksi YOLO
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        confidences = result.boxes.conf.cpu().numpy()
        class_ids = result.boxes.cls.cpu().numpy().astype(int)
        
        for i, (box, conf, cls_id) in enumerate(zip(boxes, confidences, class_ids)):
            if conf < CONFIDENCE_THRESHOLD or cls_id != 0:
                continue
                
            # Proses deteksi helm
            label_info = process_detection(frame, box, cls_id, conf)
            if not label_info:
                continue
                
            # Tambahkan variabel untuk confidence score CNN
            label, label_idx, cnn_conf = label_info
            
            # Gambar bounding box dan label
            color = (0, 0, 255) if label_idx == 1 else (0, 255, 0)
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # Update text untuk menampilkan confidence dari CNN
            text = f"{label} YOLO:{conf:.2f} CNN:{cnn_conf:.2f}"
            cv2.putText(frame, 
                       text,
                       (x1, y1 - 10),
                       cv2.FONT_HERSHEY_SIMPLEX,
                       0.6, 
                       color,
                       2)
    
    cv2.imshow("E-Tilang YOLO", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Pelanggaran terdeteksi: pelanggar\violation_20250323_213545142530.jpg (Confidence: 0.97)
Pelanggaran terdeteksi: pelanggar\violation_20250323_213547973327.jpg (Confidence: 1.00)
Pelanggaran terdeteksi: pelanggar\violation_20250323_213550334536.jpg (Confidence: 0.99)
Pelanggaran terdeteksi: pelanggar\violation_20250323_213552097456.jpg (Confidence: 0.99)
Pelanggaran terdeteksi: pelanggar\violation_20250323_213553915112.jpg (Confidence: 1.00)
Pelanggaran terdeteksi: pelanggar\violation_20250323_213555044622.jpg (Confidence: 0.99)
Pelanggaran terdeteksi: pelanggar\violation_20250323_213556851704.jpg (Confidence: 1.00)
Pelanggaran terdeteksi: pelanggar\violation_20250323_213558564712.jpg (Confidence: 1.00)
